In [1]:
import gradio as gr
import xarray as xr
import tempfile
import shutil
import os
import joblib
import xgboost
import pandas as pd
import math
import numpy as np
from tensorflow.keras.models import load_model
import io
from PIL import Image
import matplotlib.pyplot as plt

C:\Users\Usuario\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
home_md = """
# Demo TFG - Aprendizaje Automático con Datos Meteorológicos

Bienvenido/a a la demo interactiva del Trabajo de Fin de Grado.

Este entorno ha sido desarrollado como parte de la presentación del TFG y permite explorar el funcionamiento de los modelos o soluciones propuestas de forma interactica.

## Estructura de la demo

La aplicación cuenta con tres pestañas principales:

- **Home:** Esta explicación general.
- **Cuestión 1:** Demostración interactiva relacionada con un Modelo de Regresión de la parte de Modelos Tradicionales.
- **Cuestión 2:** Demostración correspondiente a un Modelos de Red Neuronal de la parte de Aplicación de Redes Neuronales.

---

> *Desarrollado por Adriana Gordillo Melero, Universidad de Sevilla, 2025*
"""

In [3]:
c1_md= """
# Aplicación de Modelo de Regresión en Datos Tabulares

En esta pestaña se verá la aplicación del modelo XGBoost para la predicción del número de rayos.

Para ello se presenta una pequeña muestra del dataset para poder realizar predicciones con la entrada seleccionada.
"""

In [4]:
c2_md = """
# Aplicación de U-Net para Predicción de rayos sobre píxeles de la Imagen

En esta pestaña se verá la aplicación del modelo de U-Net para la predicción de la probabilidad de existencia de un rayo sobre cada píxel de una imagen.

Para ello se presenta dos pestañas donde se cargarán las entradas de la Red y la salida original de dischos datos de forma respectiva.
"""

In [5]:
#Colores y diseño interfaz:
theme = gr.themes.Soft(primary_hue="blue").set(
    body_background_fill="#1e1e1e",
    body_text_color="#d4d4d4",
    input_background_fill="#2c2c2c",
    panel_background_fill="#2c2c2c",
)

# CSS adicional para forzar texto negro en dataframes
custom_css = """
/* Fuerza texto negro en celdas de Dataframe */
.gradio-container .dataframe td,
.gradio-container .dataframe th {
    color: #000 !important;
}

/* Fondo y borde del dataframe */
.gradio-container .dataframe {
    background-color: #f8f9fa !important;
    border: 1px solid #ced4da !important;
}
.gradio-container .dataframe td,
.gradio-container .dataframe th,
.gradio-container .dataframe-input td,
.gradio-container .dataframe-input th {
    color: black !important;
}

.gradio-container .dataframe,
.gradio-container .dataframe-input {
    background-color: #f8f9fa !important;
    border: 1px solid #ced4da !important;
}

/* Estilo específico para el DataFrame con ID selector-df */
#selector-df td,
#selector-df th {
    color: black !important;
}
#selector-df {
    background-color: #f8f9fa !important;
    border: 1px solid #ced4da !important;
}

/* Cambia el fondo del label para el selector */
#selector-df label {
    background-color: transparent !important;
    color: #d4d4d4 !important;
    padding: 4px 0;
}
/* Elimina fondo blanco del label del componente selector-df */
#selector-df > div:first-child {
    background-color: transparent !important;
    color: #d4d4d4 !important;
    padding: 4px 0;
}
#selector-df .label {
    background-color: transparent !important;
    color: #d4d4d4 !important;
    font-weight: bold;
}

"""


In [6]:
# Cuestión 1
salidas = pd.read_csv("examples/salidas_reg10.csv", header=None)
df_c1 = pd.read_csv("examples/entradas_reg10.csv", header=None)
xgb_model_path = "models/modelo_xgboost_reg.pkl"
xgb_model = joblib.load(xgb_model_path)

C:\Users\Usuario\anaconda3\envs\tf\lib\pickle.py:1718: UserWarning: [17:16:22] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\data\../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


In [7]:
# Cuestión 2
modelo_c2 = load_model('models/modelo_u_net_clas.h5') # quita el compile false en tu versión

In [8]:
import gradio as gr
import pandas as pd
import numpy as np
import joblib
import math
import io
from PIL import Image
import matplotlib.pyplot as plt

# Cargar datos
salidas = pd.read_csv("examples/salidas_reg10.csv", header=None)
df_c1 = pd.read_csv("examples/entradas_reg10.csv", header=None)
xgb_model_path = "models/modelo_xgboost_reg.pkl"
xgb_model = joblib.load(xgb_model_path)

# Función para Cuestión 1
def estimar_cuestion1(df_predict: pd.DataFrame, index: float) -> str:
    y_pred = xgb_model.predict(df_predict)
    index = int(index)
    valor_real = salidas.iloc[index, 0]
    text = f"Número real de rayos en la imagen: {valor_real}\n"
    text += f"Número de rayos predecidos por el modelo: {math.ceil(y_pred[0])}"
    return text

# Funciones para Cuestión 2
def load_npy_file(file):
    return np.load(file.name)

def estimar_cuestion2(file: gr.File):
    example = load_npy_file(file)
    resultado = modelo_c2.predict(example)[0]
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(resultado)
    ax.axis('off')
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)

def mostrar_imagen_npy(file):
    example = load_npy_file(file)
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(example[0])
    ax.axis('off')
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)

def mostrar_datos_npy_con_canal(file, canal):
    try:
        data = np.load(file.name)[0]
        if data.ndim == 3 and data.shape[-1] > 1:
            if 0 <= canal < data.shape[-1]:
                matriz = data[:, :, canal]
            else:
                return pd.DataFrame([["Canal fuera de rango."]])
        elif data.ndim == 2 or (data.ndim == 3 and data.shape[-1] == 1):
            matriz = data[:, :, 0] if data.ndim == 3 else data
        else:
            return pd.DataFrame([["Formato no soportado."]])
        return pd.DataFrame(matriz)
    except Exception as e:
        return pd.DataFrame([["Error:", str(e)]])

def mostrar_datos_npy(file):
    data = np.load(file)
    return pd.DataFrame(data[0])

# # Interfaz Gradio


def estilo_negro(df):
    return df.style.set_properties(**{'color': 'black'})

def actualizar_fila(evt: gr.SelectData):
    index = evt.index[0]
    return estilo_negro(df_c1.iloc[[index]]), index

def actualizar_entrada(file, canal_str):
    canal = int(canal_str)
    df = mostrar_datos_npy_con_canal(file, canal)
    return estilo_negro(df)

def actualizar_salida(file):
    df = mostrar_datos_npy(file)
    return estilo_negro(df)

demo = gr.Blocks(
    theme=theme, css=custom_css,
    title="TFG Demo"
)

with demo:
    with gr.Tabs():
        with gr.Tab("Home"):
            gr.Markdown(home_md)

        with gr.Tab("Cuestión 1"):
            gr.Markdown(c1_md)
            with gr.Row():
                with gr.Column():
                    df_selector = gr.DataFrame(
                        value=df_c1,
                        col_count=df_c1.shape[1],
                        interactive=True,
                        label="Selecciona una fila del dataset",
                        type="pandas",
                        elem_id="selector-df"
                    )


                    selected_row = gr.DataFrame(
                        value=estilo_negro(df_c1.iloc[[0]]),
                        col_count=df_c1.shape[1],
                        interactive=False,
                        label="Fila seleccionada para predecir",
                        type="pandas"
                    )

                    selected_index = gr.Number(visible=False)

                with gr.Column():
                    output_c1 = gr.Textbox(label="Resultado de la predicción", lines=4)
                    with gr.Row(variant='panel'):
                        predict_btn = gr.Button("Predecir")
                        predict_btn.click(
                            fn=estimar_cuestion1,
                            inputs=[selected_row, selected_index],
                            outputs=output_c1
                        )

            df_selector.select(fn=actualizar_fila, inputs=None, outputs=[selected_row, selected_index])

        with gr.Tab("Cuestión 2"):
            gr.Markdown(c2_md)

            with gr.Row():
                with gr.Column(scale=1):
                    file_input1 = gr.File(label="Sube tus entradas del modelo (.npy)", file_types=[".npy"], file_count="single")
                    file_input2 = gr.File(label="Sube tu archivo salida original (.npy)", file_types=[".npy"], file_count="single")

                    canal_selector = gr.Dropdown(
                        choices=["0", "1", "2", "3"],
                        label="Selecciona el canal del predictor",
                        value="0"
                    )

                    gr.Markdown("#### Matrices cargadas")
                    with gr.Row():
                        data_input1 = gr.DataFrame(label="Canal seleccionado", interactive=False)
                        data_input2 = gr.DataFrame(label="Salida original", interactive=False)

                with gr.Column(scale=1):
                    imagen_mostrada = gr.Image(label="Vista previa del modelo")
                    output_image = gr.Image(label="Imagen generada por el modelo")
                    with gr.Row(variant='panel'):
                        predict_btn2 = gr.Button("Predecir")
                        predict_btn2.click(fn=estimar_cuestion2, inputs=[file_input1], outputs=output_image)

            file_input1.change(fn=actualizar_entrada, inputs=[file_input1, canal_selector], outputs=[data_input1])
            canal_selector.change(fn=actualizar_entrada, inputs=[file_input1, canal_selector], outputs=[data_input1])
            file_input2.change(fn=actualizar_salida, inputs=[file_input2], outputs=[data_input2])
            file_input2.change(fn=mostrar_imagen_npy, inputs=[file_input2], outputs=[imagen_mostrada])

demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://04a6fca220caf6afc9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 0s 272ms/step


Traceback (most recent call last):
  File "C:\Users\Usuario\anaconda3\envs\tf\lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "C:\Users\Usuario\anaconda3\envs\tf\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "C:\Users\Usuario\anaconda3\envs\tf\lib\site-packages\gradio\blocks.py", line 2146, in process_api
    result = await self.call_function(
  File "C:\Users\Usuario\anaconda3\envs\tf\lib\site-packages\gradio\blocks.py", line 1664, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "C:\Users\Usuario\anaconda3\envs\tf\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "C:\Users\Usuario\anaconda3\envs\tf\lib\site-packages\anyio\_backends\_asyncio.py", line 2505, in run_sync_in_worker_thread
    return await future


1/1 [==============================] - 0s 20ms/step


Traceback (most recent call last):
  File "C:\Users\Usuario\anaconda3\envs\tf\lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "C:\Users\Usuario\anaconda3\envs\tf\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "C:\Users\Usuario\anaconda3\envs\tf\lib\site-packages\gradio\blocks.py", line 2146, in process_api
    result = await self.call_function(
  File "C:\Users\Usuario\anaconda3\envs\tf\lib\site-packages\gradio\blocks.py", line 1664, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "C:\Users\Usuario\anaconda3\envs\tf\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "C:\Users\Usuario\anaconda3\envs\tf\lib\site-packages\anyio\_backends\_asyncio.py", line 2505, in run_sync_in_worker_thread
    return await future


1/1 [==============================] - 0s 15ms/step
